In [1]:
# !pip install openai

In [2]:
import pandas as pd
from openai import OpenAI
import json
import time
import re
import os
from dotenv import load_dotenv


# Load API key
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
df = pd.read_csv("data.csv")      # datasetnya diduplikat trs diganti aja namanya jd ini

In [4]:
# df.drop("sentimen", axis=1, inplace=True)

In [5]:
if "sentimen" not in df.columns:
    df["sentimen"] = ""

In [6]:
unlabeled = df[df["sentimen"].isna() | (df["sentimen"] == "")]
batch_size = 10     # ini biar sekali ngeprompt ngambil banyak data dan dapat sentimennya sebanyak data itu. biar ga 1 data 1 prompt, keberatan.
model_gpt = "gpt-5-nano"

topik = "gojek"

In [8]:
for start in range(0, len(unlabeled), batch_size):
    batch = unlabeled.iloc[start:start+batch_size]

    texts = batch["full_text"].tolist()
    indexed_texts = "\n".join([f"{i+1}. {t}" for i, t in enumerate(texts)])

    prompt = f"""
    Kamu adalah model analisis sentimen. topik pembahasannya adalah "{topik}"
    Tentukan sentimen untuk setiap teks berikut (positif, netral, dan negatif).
    Balas DALAM FORMAT JSON PLAIN, tanpa tanda ``` atau teks lain.
    Contoh:
    {{"1": "positif", "2": "negatif"}}

    Teks:
    {indexed_texts}
    """

    try:
        response = client.chat.completions.create(
            model= model_gpt,
            messages=[{"role": "user", "content": prompt}],
        )

        result_text = response.choices[0].message.content.strip()

        # 💡 Bersihin kalau model nambahin ```json ... ```
        result_text = re.sub(r"^```(?:json)?|```$", "", result_text, flags=re.MULTILINE).strip()

        try:
            sentiments = json.loads(result_text)
        except json.JSONDecodeError:
            print("⚠️ Gagal parse JSON, hasil mentah:", result_text)
            continue

        for i, (_, row) in enumerate(batch.iterrows()):
            df.at[row.name, "sentimen"] = sentiments.get(str(i+1), "")

        df.to_csv("data.csv", index=False)
        print(f"✅ Batch {start}-{start+batch_size} selesai")

        time.sleep(2)

    except Exception as e:
        err = str(e)
        if "429" in err:
            print("⚠️ Kena rate limit, tidur dulu 25 detik...")
            time.sleep(25)
            continue
        else:
            print(f"❌ Error batch {start}: {e}")
            time.sleep(5)

✅ Batch 0-10 selesai
✅ Batch 10-20 selesai
✅ Batch 20-30 selesai
✅ Batch 30-40 selesai
✅ Batch 40-50 selesai
✅ Batch 50-60 selesai
✅ Batch 60-70 selesai
✅ Batch 70-80 selesai
✅ Batch 80-90 selesai
✅ Batch 90-100 selesai


In [ ]:
positif_count = (df["sentimen"] == "positif").sum()
negatif_count = (df["sentimen"] == "negatif").sum()

print("Jumlah positif:", positif_count)
print("Jumlah negatif:", negatif_count)